# Process Vectrino output to get averaged statistics

This notebook takes Vectrino generated ASCII time series and with name of the form ```Run01_01_*.dat```, and filters, computes time series mean and Reynold stress values, and outputs a summary table of the mean values. It works on all files in a director.

Needs:
- Vectrino ```*.dat``` ASCII time series data

User input:
- Data folder
- Frequency at which the data was collected, e.g. 25 Hz
- SNR and COR filter criteria

Version/Change log:
- V01 initial build 2/22/22 KS

## Loads

In [4]:
# %pylab inline
import numpy as np
from numpy import log, sin, cos, exp, tan, pi
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

%matplotlib inline
%config InlineBackend.figure_format='retina' # hig-res plots for a Retina display, uncomment while working

# uncomment the two lines below when exporting to a pdf
# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('png','pdf')

## Find the data files and prep for loading the files 

In [5]:
""" --- User input ----------------------------- """

fs = 25 # frequency in Hz that the data was recorded at

cf = 70   # correlation filter, drop everything below
snrf = 10 # snr filter, drop everything below this value

# datadir = 'ADV_test/' # path to the data
datadir = 'test_data/' # path to the data

"""  ------------------------------------------- """

ext = '*.dat'     # data file extension

# Specify the column header names in the *.dat files. These names can be found in the .hrd file.

cnames = ['time_s','status','v1_m_s', 'v2_m_s','v3_m_s','v4_m_s','amp1', 'amp2','amp3','amp4',
         'snr1', 'snr2','snr3','snr4','cor1', 'cor2','cor3','cor4']

# Find all of the data files and store the names 

files = sorted(glob.glob(datadir+ext))
nf=len(files)
os.chdir(datadir)
filenames = sorted(glob.glob(ext))
for i in range(0,nf):
    filenames[i]=os.path.splitext(os.path.basename(filenames[i]))[0] #strips extension
os.chdir("..")
    
#strip the endings 

datapt=filenames.copy()
run=filenames.copy()
for i in range(0,len(filenames)):
    stname = filenames[i]    # file names
    run[i]=stname[0:5]       # run 
    datapt[i]=stname[6:8]    # ADV point in run
    
#Print the file names and index value for each file

for i in range(0,nf):
    print('file: ',filenames[i],'(index: ',i,')')

file:  Run1B_01 (index:  0 )
file:  Run1B_02 (index:  1 )
file:  Run1B_03 (index:  2 )
file:  Run1B_04 (index:  3 )
file:  Run1B_05 (index:  4 )
file:  Run1B_06 (index:  5 )
file:  Run1B_07 (index:  6 )
file:  Run1B_08 (index:  7 )
file:  Run1B_09 (index:  8 )
file:  Run1B_10 (index:  9 )
file:  Run1B_11 (index:  10 )


## Process and summarize all data in profile

In [6]:
uu_bar = np.zeros(nf)
uv_bar = np.zeros(nf)
uw_bar = np.zeros(nf)
vv_bar = np.zeros(nf)
vw_bar = np.zeros(nf)
ww_bar = np.zeros(nf)

for j in range(nf):
    
    # load up the file
    data = pd.read_csv(files[j], sep='\s+', header=None) # reads in the data
    data.columns = cnames # renames the columns
    data.iloc[:,0]=np.arange(len(data))/fs # changes the counter to time
    data=data.drop(['status'], axis=1)  # delete the "status" column
    
    # filter based on cor and snr
    data = data[(data.cor1 >= cf)&(data.cor2 >= cf)&(data.cor3 >= cf)&(data.cor3 >= cf)
            &(data.snr1 >= snrf)&(data.snr2 >= snrf)&(data.snr3 >= snrf)&(data.snr4 >= snrf)].copy()
    
    # calculate the means and store in a new dataframe
    if j == 0:
        summary = pd.DataFrame([data.mean()])
    else:
        summary.loc[len(summary.index)]=data.mean()
        
    # calculate the turbulent statistics
    up = np.array(data.v1_m_s-summary.v1_m_s[j])
    vp = np.array(data.v2_m_s-summary.v2_m_s[j])
    wp = np.array(data.v3_m_s-summary.v3_m_s[j])

    uu_bar[j] = np.mean(up*up)
    uv_bar[j] = np.mean(up*vp)
    uw_bar[j] = np.mean(up*wp)
    vv_bar[j] = np.mean(vp*vp)
    vw_bar[j] = np.mean(vp*wp)
    ww_bar[j] = np.mean(wp*wp)
    
# clean up the dataframe column names and add in the z values
summary.rename(columns={'time_s': 'Run'},inplace=True)
summary['Run']=run
summary.rename(columns={'v1_m_s': 'uavg_m_s', 'v2_m_s': 'vavg_m_s', 'v3_m_s': 'wavg_m_s'},inplace=True)
summary = summary.drop(columns=['v4_m_s'])
summary = summary.drop(columns=['amp1'])
summary = summary.drop(columns=['amp2'])
summary = summary.drop(columns=['amp3'])
summary = summary.drop(columns=['amp4'])
summary.insert(loc=1, column='Point', value=datapt)
summary.insert(loc=5, column='uu_bar', value=uu_bar)
summary.insert(loc=6, column='uv_bar', value=uv_bar)
summary.insert(loc=7, column='uw_bar', value=uw_bar)
summary.insert(loc=8, column='vv_bar', value=vv_bar)
summary.insert(loc=9, column='vw_bar', value=vw_bar)
summary.insert(loc=10, column='ww_bar', value=ww_bar)

# save the summary file as a csv
summary.to_csv(datadir+'summary_velocity_mean.csv',index=False)

# print out the summary file
summary

,Run,Point,uavg_m_s,vavg_m_s,wavg_m_s,uu_bar,uv_bar,uw_bar,vv_bar,vw_bar,ww_bar,snr1,snr2,snr3,snr4,cor1,cor2,cor3,cor4
0,Run1B,01,0.508021,-0.015163,-0.006950,0.013791,-0.002389,-0.003682,0.004937,0.001430,0.007298,18.651585,21.108743,15.822623,15.765246,80.323497,80.427322,79.989071,78.488525
1,Run1B,02,0.578202,-0.003716,-0.009438,0.014849,-0.000828,-0.004090,0.004062,0.000463,0.007116,18.663321,21.114686,15.837122,15.757269,81.775646,81.966790,80.592620,80.538007
2,Run1B,03,0.641050,-0.002523,-0.010512,0.012055,-0.001475,-0.003125,0.003970,0.000676,0.005598,18.662935,21.112507,15.837407,15.764306,82.431182,83.056539,81.330668,81.260994
3,Run1B,04,0.701200,0.003747,-0.020070,0.010088,-0.001173,-0.002476,0.003661,0.000246,0.005176,18.662267,21.110010,15.837684,15.762022,83.650147,84.317959,82.864573,82.218842
4,Run1B,05,0.779713,-0.000025,-0.027955,0.008809,-0.000826,-0.001892,0.002521,0.000229,0.003819,18.664558,21.115220,15.844421,15.761608,85.416845,85.902950,84.557931,84.734074
5,Run1B,06,0.853329,-0.007601,-0.046137,0.005580,-0.000504,-0.001427,0.001904,0.000352,0.012561,18.665824,21.114598,15.836475,14.979693,87.985441,88.165134,86.896169,77.106897
6,Run1B,07,0.620484,0.022383,0.056355,0.010572,-0.000154,-0.001848,0.003010,0.000225,0.004831,18.654204,21.103347,15.845020,15.752653,85.489796,85.936327,84.805714,84.132653
7,Run1B,08,0.810672,0.056857,0.124392,0.005420,-0.000353,0.000310,0.002744,0.000485,0.003146,18.646386,21.101635,15.836360,15.746256,84.309811,84.952668,84.165663,82.887263
8,Run1B,09,0.980984,0.012623,0.073426,0.004257,0.000907,0.001067,0.002353,-0.001701,0.155131,18.649180,21.091803,15.819672,13.557377,77.770492,77.459016,79.754098,29.737705
9,Run1B,10,-0.201621,-0.017909,0.056727,0.009250,0.001457,-0.001828,0.005978,-0.000400,0.011214,18.634361,21.089676,15.814638,15.735439,83.850539,83.243451,82.746533,81.138675
